In [ ]:
pip install gensim

In [ ]:
pip install google_play_scraper

In [ ]:
import pandas as pd
from google_play_scraper import reviews, Sort

result, _ = reviews(
    'com.tokopedia.tkpd',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count=15000,
    filter_score_with=None
)

data_ulasan = []
for r in result:
    data_ulasan.append({
        'content': r['content'],
        'score': r['score']
    })

df_ulasan = pd.DataFrame(data_ulasan)
print("Contoh Data dengan Skor:")
print(df_ulasan.head())
print("\nDistribusi Skor Bintang:")
print(df_ulasan['score'].value_counts())

Contoh Data dengan Skor:
                                             content  score
0                                                 ok      5
1  Kita pengen tokopedia yg dulu, tokopedia gausa...      1
2                                            terbaik      5
3  Setiap bulan saya pesan brang ditokopedia sela...      1
4  aplikasi paling rajin update sedunia,seminggu ...      1

Distribusi Skor Bintang:
score
5    5403
1    4399
4     790
2     737
3     671
Name: count, dtype: int64


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
pip install sastrawi


In [ ]:
import pandas as pd

# Load CSV kamus alay
kamus_df = pd.read_csv('kamusalay.csv', names=['alay', 'baku'], encoding='utf-8')

In [ ]:
kamus_df.head()

,alay,baku
0,ajh,saja
1,ajj,saja
2,akoh,aku
3,amaca,ah masa
4,amuh,kamu


In [ ]:
alay_dict = dict(zip(kamus_df['alay'], kamus_df['baku']))

In [ ]:
pip install swifter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=56972d806dca003559ddb76e8762c1d7ef33a58b8aef9e46b0332b979f09f348
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm

# Aktifkan progress_apply untuk tqdm
tqdm.pandas()

# ========== 2. Inisialisasi Sekali ==========
stop_words = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# ========== 3. Normalisasi Slang ==========
def normalize_slang_from_dict(text, slang_dict):
    words = text.split()
    return ' '.join([slang_dict.get(word, word) for word in words])

# ========== 4. Fungsi Clean Text ==========
def clean_text(text, slang_dict):
    if pd.isna(text):
        return ''

    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Normalisasi slang
    text = normalize_slang_from_dict(text, slang_dict)

    # Stopword removal
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Stemming
    words = [stemmer.stem(word) for word in words]

    return ' '.join(words)

import swifter
df_ulasan['cleaned_content'] = df_ulasan['content'].swifter.apply(lambda x: clean_text(x, alay_dict))


print("\nContoh Data Setelah Preprocessing:")
print(df_ulasan.head())

Pandas Apply:   0%|          | 0/12000 [00:00<?, ?it/s]


Contoh Data Setelah Preprocessing:
                                             content  score  \
0                                                 ok      5   
1  Kita pengen tokopedia yg dulu, tokopedia gausa...      1   
2                                            terbaik      5   
3  Setiap bulan saya pesan brang ditokopedia sela...      1   
4  aplikasi paling rajin update sedunia,seminggu ...      1   

                                     cleaned_content  
0                                                oke  
1  tokopedia yg tokopedia gausah gabung in tiktok...  
2                                               baik  
3  pesan brang ditokopedia sja barang yg dtangny ...  
4       aplikasi rajin update seduniaseminggu update  


In [ ]:
df_ulasan.to_csv('preprocessing_review_tokepedia.csv', index=False)

In [ ]:
import pandas as pd

# Ubah sesuai path file kamu
positive_words = pd.read_csv('positive.tsv', sep='\t', header=None)[0].tolist()
negative_words = pd.read_csv('negative.tsv', sep='\t', header=None)[0].tolist()

# Ubah ke lowercase untuk nyocokin
positive_words = [word.lower() for word in positive_words]
negative_words = [word.lower() for word in negative_words]


In [ ]:
def label_sentiment(text):
    if pd.isna(text):
        return 'netral'

    words = text.split()
    pos = sum(1 for word in words if word in positive_words)
    neg = sum(1 for word in words if word in negative_words)

    if pos > neg:
        return 'positif'
    elif neg > pos:
        return 'negatif'
    else:
        return 'netral'


In [ ]:
df_ulasan['sentiment'] = df_ulasan['cleaned_content'].apply(label_sentiment)

# Cek hasil
print(df_ulasan[['cleaned_content', 'sentiment']].head())


                                     cleaned_content sentiment
0                                                oke   positif
1  tokopedia yg tokopedia gausah gabung in tiktok...   negatif
2                                               baik    netral
3  pesan brang ditokopedia sja barang yg dtangny ...   negatif
4       aplikasi rajin update seduniaseminggu update   negatif


In [ ]:
df_ulasan.to_csv('labeling_review_tokepedia.csv', index=False)

In [ ]:
print(df_ulasan['sentiment'].value_counts())


sentiment
positif    5502
netral     3883
negatif    2615
Name: count, dtype: int64


In [2]:
import pandas as pd

In [3]:
df_ulasan = pd.read_csv('labeling_review_tokepedia.csv')

In [ ]:
print(df_ulasan.head())

                                             content  score  \
0                                                 ok      5   
1  Kita pengen tokopedia yg dulu, tokopedia gausa...      1   
2                                            terbaik      5   
3  Setiap bulan saya pesan brang ditokopedia sela...      1   
4  aplikasi paling rajin update sedunia,seminggu ...      1   

                                     cleaned_content sentiment  
0                                                oke   positif  
1  tokopedia yg tokopedia gausah gabung in tiktok...   negatif  
2                                               baik    netral  
3  pesan brang ditokopedia sja barang yg dtangny ...   negatif  
4       aplikasi rajin update seduniaseminggu update   negatif  


In [ ]:
print(df_ulasan.isnull().sum())

content              1
score                0
cleaned_content    263
sentiment            0
dtype: int64


In [4]:
#hapus data yang null
df_ulasan.dropna(inplace=True)

In [ ]:
print(df_ulasan.isnull().sum())

content            0
score              0
cleaned_content    0
sentiment          0
dtype: int64


In [ ]:
df_ulasan.duplicated().sum()

1846

In [5]:
#drop duplikat
df_ulasan.drop_duplicates(inplace=True)

In [ ]:
df_ulasan.duplicated().sum()

0

In [6]:
print(df_ulasan['sentiment'].value_counts())

sentiment
positif    4409
netral     2890
negatif    2592
Name: count, dtype: int64


In [7]:
pip install imbalanced-learn

In [8]:
X = df_ulasan['cleaned_content']
y = df_ulasan['sentiment']

In [9]:
# from sklearn.model_selection import train_test_split
# # Bagi data menjadi Training + Validation Set (80%) dan Testing Set (20%)
# X_train_val, X_test, y_train_val, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # Bagi Training + Validation Set (yang 80%) menjadi Training Set (60%) dan Validation Set (20%)
# # test_size=0.25 dari X_train_val (0.25 * 0.8 = 0.2 dari total)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
# )
from sklearn.model_selection import train_test_split

# Bagi data menjadi Training Set (80%) dan Testing Set (20%) saja
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [10]:
print("\nDistribusi kelas di Training Set:")
print(y_train.value_counts())
# print("\nDistribusi kelas di Validation Set:")
# print(y_val.value_counts())
print("\nDistribusi kelas di Testing Set:")
print(y_test.value_counts())


Distribusi kelas di Training Set:
sentiment
positif    3527
netral     2312
negatif    2073
Name: count, dtype: int64

Distribusi kelas di Testing Set:
sentiment
positif    882
netral     578
negatif    519
Name: count, dtype: int64


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5, max_df=0.8, ngram_range=(1, 2))

# Fit HANYA pada data training, kemudian transform pada semua set
X_train_vec = vectorizer.fit_transform(X_train)
# X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)

In [12]:
from imblearn.over_sampling import SMOTE
import numpy as np
from scipy.sparse import csr_matrix


In [13]:
print("\nDistribusi kelas di Training Set sebelum balancing:")
print(y_train.value_counts())


Distribusi kelas di Training Set sebelum balancing:
sentiment
positif    3527
netral     2312
negatif    2073
Name: count, dtype: int64


In [14]:


# Strategi SMOTE: hanya oversample kelas 'Netral' ke 536 sampel
# sampling_strategy_smote = {
#     'netral': 3527,
#     'negatif': 3527
# }

sampling_strategy_smote = {
    'netral': 6000,
    'negatif': 6000,
    'positif':6000
}

# Inisialisasi SMOTE
smote_sampler = SMOTE(sampling_strategy=sampling_strategy_smote, random_state=42)

# Lakukan resampling hanya dengan SMOTE
X_train_resampled_tfidf, y_train_resampled_tfidf = smote_sampler.fit_resample(X_train_vec, y_train)

# Cek hasil
print("\nDistribusi kelas di Training Set setelah SMOTE:")
print(y_train_resampled_tfidf.value_counts())
print(f"Total sampel training setelah balancing: {len(y_train_resampled_tfidf)}")



Distribusi kelas di Training Set setelah SMOTE:
sentiment
negatif    6000
positif    6000
netral     6000
Name: count, dtype: int64
Total sampel training setelah balancing: 18000


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
import time

# === 1. Tokenisasi data (list of list of tokens untuk Word2Vec) ===
X_train_tokenized = [text.split() for text in X_train]
# X_val_tokenized = [text.split() for text in X_val]
X_test_tokenized = [text.split() for text in X_test]

print(f"Contoh tokenisasi pertama: {X_train_tokenized[0]}")

# === 2. Latih Word2Vec dari data training lokal ===
print("\nMelatih Word2Vec dari data training...")
start_time = time.time()

w2v_model = Word2Vec(
    sentences=X_train_tokenized,
    vector_size=100,  # Dimensi embedding
    window=5,
    min_count=1,
    sg=0,  # CBOW (lebih cepat)
    workers=-1,
    epochs=10
)

end_time = time.time()
print(f"Model Word2Vec selesai dalam {end_time - start_time:.2f} detik")
print(f"Total kata dalam vocabulary Word2Vec: {len(w2v_model.wv.index_to_key)}")

# === 3. Fungsi untuk merata-rata embedding Word2Vec per kalimat ===
def sentence_vector(tokens, model, vector_size):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(vectors, axis=0)

# === 4. Buat vektor rata-rata untuk X_train, X_val, X_test ===
print("\nMembuat representasi rata-rata Word2Vec...")
X_train_w2v = np.array([sentence_vector(tokens, w2v_model, 100) for tokens in X_train_tokenized])
# X_val_w2v = np.array([sentence_vector(tokens, w2v_model, 100) for tokens in X_val_tokenized])
X_test_w2v = np.array([sentence_vector(tokens, w2v_model, 100) for tokens in X_test_tokenized])

print("Shape X_train_w2v:", X_train_w2v.shape)

# === 5. Oversampling dengan SMOTE ===
print("\nDistribusi awal y_train:")
print(y_train.value_counts())

sampling_strategy = {
    'netral': 3527,
    'negatif': 3527
    # Positif tidak diubah (sudah 2645)
}
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)

X_train_w2v_resampled, y_train_resampled = smote.fit_resample(X_train_w2v, y_train)

print("\nDistribusi kelas setelah oversampling:")
print(pd.Series(y_train_resampled).value_counts())
print(f"Total sampel setelah oversampling: {len(y_train_resampled)}")


Contoh tokenisasi pertama: ['versi', 'sayang', 'masuk', 'pakai', 'akun']

Melatih Word2Vec dari data training...


Model Word2Vec selesai dalam 0.26 detik
Total kata dalam vocabulary Word2Vec: 8709

Membuat representasi rata-rata Word2Vec...
Shape X_train_w2v: (7912, 100)

Distribusi awal y_train:
sentiment
positif    3527
netral     2312
negatif    2073
Name: count, dtype: int64

Distribusi kelas setelah oversampling:
sentiment
negatif    3527
positif    3527
netral     3527
Name: count, dtype: int64
Total sampel setelah oversampling: 10581


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, f1_score
import time # Untuk mengukur waktu pelatihan

print("--- Memulai Pelatihan dan Evaluasi Model Klasifikasi ---")

# --- Model 1: Random Forest Classifier ---
print("\nMelatih Random Forest Classifier...")
start_time = time.time()
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 untuk gunakan semua core CPU
rf_model.fit(X_train_resampled_tfidf, y_train_resampled_tfidf)
end_time = time.time()
print(f"Waktu pelatihan Random Forest: {end_time - start_time:.2f} detik")

# Prediksi dan Evaluasi pada Validation Set
# y_val_pred_rf = rf_model.predict(X_val_vec)
# print("\n--- Hasil Random Forest pada Validation Set (TF-IDF) ---")
# print(classification_report(y_val, y_val_pred_rf))
# print(f"Accuracy: {accuracy_score(y_val, y_val_pred_rf):.4f}")
# print(f"F1-Score (weighted): {f1_score(y_val, y_val_pred_rf, average='weighted'):.4f}")

# Prediksi dan Evaluasi pada Test Set (Evaluasi Akhir)
y_test_pred_rf = rf_model.predict(X_test_vec)
print("\n--- Hasil Random Forest pada Test Set (TF-IDF, Final) ---")
print(classification_report(y_test, y_test_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_rf):.4f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_test_pred_rf, average='weighted'):.4f}")


# --- Model 2: Support Vector Classifier (SVC) ---
print("\nMelatih Support Vector Classifier (SVC)...")
start_time = time.time()
# kernel='linear' sering efektif untuk data teks sparse seperti TF-IDF.
# C adalah parameter regulerisasi, bisa disetel nanti.
svc_model = SVC(kernel='linear', random_state=42)
svc_model.fit(X_train_resampled_tfidf, y_train_resampled_tfidf)
end_time = time.time()
print(f"Waktu pelatihan SVC: {end_time - start_time:.2f} detik")

# Prediksi dan Evaluasi pada Validation Set
# y_val_pred_svc = svc_model.predict(X_val_vec)
# print("\n--- Hasil SVC pada Validation Set (TF-IDF) ---")
# print(classification_report(y_val, y_val_pred_svc))
# print(f"Accuracy: {accuracy_score(y_val, y_val_pred_svc):.4f}")
# print(f"F1-Score (weighted): {f1_score(y_val, y_val_pred_svc, average='weighted'):.4f}")

# Prediksi dan Evaluasi pada Test Set (Evaluasi Akhir)
y_test_pred_svc = svc_model.predict(X_test_vec)
print("\n--- Hasil SVC pada Test Set (TF-IDF, Final) ---")
print(classification_report(y_test, y_test_pred_svc))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_svc):.4f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_test_pred_svc, average='weighted'):.4f}")


from sklearn.preprocessing import LabelEncoder
import xgboost as xgb # Pastikan ini sudah diimpor
from sklearn.metrics import classification_report, accuracy_score, f1_score
import time # Untuk mengukur waktu pelatihan

# --- Asumsi: X_train_resampled_tfidf, y_train_resampled_tfidf, X_val_vec_tfidf, y_val, X_test_vec_tfidf, y_test sudah tersedia ---

print("\n--- Mengonversi Label Sentimen ke Numerik untuk XGBoost ---")

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder HANYA pada y_train_resampled_tfidf untuk mempelajari pemetaan
# Kemudian transform semua set label
y_train_encoded = label_encoder.fit_transform(y_train_resampled_tfidf)
# y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

print(f"Mapping label: {list(label_encoder.classes_)} -> {list(range(len(label_encoder.classes_)))}")
print(f"Contoh y_train_encoded (awal): {y_train_encoded[:5]}")
print(f"Contoh y_train_resampled_tfidf (awal): {y_train_resampled_tfidf[:5].tolist()}")


# --- Lanjutkan dengan Pelatihan XGBoost Classifier ---
print("\nMelatih XGBoost Classifier (dengan label numerik)...")
start_time = time.time()
xgb_model = xgb.XGBClassifier(objective='multi:softmax',
                              num_class=len(label_encoder.classes_), # Jumlah kelas disesuaikan
                              eval_metric='mlogloss',
                              use_label_encoder=False, # Pertahankan ini
                              n_estimators=100,
                              random_state=42,
                              n_jobs=-1)

# Gunakan label yang sudah di-encode
xgb_model.fit(X_train_resampled_tfidf, y_train_encoded)
end_time = time.time()
print(f"Waktu pelatihan XGBoost: {end_time - start_time:.2f} detik")

# Prediksi pada Validation Set (akan menghasilkan label numerik)
# y_val_pred_xgb_encoded = xgb_model.predict(X_val_vec)
# Konversi kembali prediksi ke label string untuk classification_report yang mudah dibaca
# y_val_pred_xgb = label_encoder.inverse_transform(y_val_pred_xgb_encoded)

# print("\n--- Hasil XGBoost pada Validation Set (TF-IDF) ---")
# print(classification_report(y_val, y_val_pred_xgb)) # y_val masih string
# print(f"Accuracy: {accuracy_score(y_val, y_val_pred_xgb):.4f}")
# print(f"F1-Score (weighted): {f1_score(y_val, y_val_pred_xgb, average='weighted'):.4f}")

# Prediksi pada Test Set (Evaluasi Akhir)
y_test_pred_xgb_encoded = xgb_model.predict(X_test_vec)
# Konversi kembali prediksi ke label string
y_test_pred_xgb = label_encoder.inverse_transform(y_test_pred_xgb_encoded)

print("\n--- Hasil XGBoost pada Test Set (TF-IDF, Final) ---")
print(classification_report(y_test, y_test_pred_xgb)) # y_test masih string
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_xgb):.4f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_test_pred_xgb, average='weighted'):.4f}")

--- Memulai Pelatihan dan Evaluasi Model Klasifikasi ---

Melatih Random Forest Classifier...
Waktu pelatihan Random Forest: 14.08 detik

--- Hasil Random Forest pada Test Set (TF-IDF, Final) ---
              precision    recall  f1-score   support

     negatif       0.77      0.77      0.77       519
      netral       0.80      0.67      0.73       578
     positif       0.81      0.90      0.85       882

    accuracy                           0.80      1979
   macro avg       0.79      0.78      0.78      1979
weighted avg       0.80      0.80      0.79      1979

Accuracy: 0.7969
F1-Score (weighted): 0.7939

Melatih Support Vector Classifier (SVC)...
Waktu pelatihan SVC: 27.71 detik

--- Hasil SVC pada Test Set (TF-IDF, Final) ---
              precision    recall  f1-score   support

     negatif       0.81      0.87      0.84       519
      netral       0.80      0.74      0.77       578
     positif       0.91      0.91      0.91       882

    accuracy                      

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:56:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Waktu pelatihan XGBoost: 42.42 detik

--- Hasil XGBoost pada Test Set (TF-IDF, Final) ---
              precision    recall  f1-score   support

     negatif       0.79      0.77      0.78       519
      netral       0.71      0.70      0.71       578
     positif       0.83      0.85      0.84       882

    accuracy                           0.79      1979
   macro avg       0.78      0.77      0.78      1979
weighted avg       0.78      0.79      0.79      1979

Accuracy: 0.7858
F1-Score (weighted): 0.7852


In [18]:
import joblib
# Simpan Random Forest
joblib.dump(rf_model, 'model_random_forest_tfidf.pkl')

# Simpan SVC
joblib.dump(svc_model, 'model_svc_tfidf.pkl')

# Simpan XGBoost
joblib.dump(xgb_model, 'model_xgboost_tfidf.pkl')

# Simpan LabelEncoder (penting untuk XGBoost agar bisa inverse_transform saat prediksi)
joblib.dump(label_encoder, 'label_encoder_tfidf.pkl')


['label_encoder_tfidf.pkl']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, f1_score
import time

print("\n--- Melatih dan Mengevaluasi Model Klasifikasi (Word2Vec Lokal) ---")

# ========================
# Model 1: Random Forest
# ========================
print("\nMelatih Random Forest Classifier...")
start_time = time.time()
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_w2v_resampled, y_train_resampled)
end_time = time.time()
print(f"Waktu pelatihan RF: {end_time - start_time:.2f} detik")

# y_val_pred_rf = rf_model.predict(X_val_w2v)
# print("\n[Random Forest] Validation Set:")
# print(classification_report(y_val, y_val_pred_rf))
# print(f"Accuracy: {accuracy_score(y_val, y_val_pred_rf):.4f}")
# print(f"F1-Score (weighted): {f1_score(y_val, y_val_pred_rf, average='weighted'):.4f}")

y_test_pred_rf = rf_model.predict(X_test_w2v)
print("\n[Random Forest] Test Set:")
print(classification_report(y_test, y_test_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_rf):.4f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_test_pred_rf, average='weighted'):.4f}")


# ========================
# Model 2: SVC
# ========================
print("\nMelatih Support Vector Classifier (SVC)...")
start_time = time.time()
svc_model = SVC(kernel='linear', random_state=42)
svc_model.fit(X_train_w2v_resampled, y_train_resampled)
end_time = time.time()
print(f"Waktu pelatihan SVC: {end_time - start_time:.2f} detik")

# y_val_pred_svc = svc_model.predict(X_val_w2v)
# print("\n[SVC] Validation Set:")
# print(classification_report(y_val, y_val_pred_svc))
# print(f"Accuracy: {accuracy_score(y_val, y_val_pred_svc):.4f}")
# print(f"F1-Score (weighted): {f1_score(y_val, y_val_pred_svc, average='weighted'):.4f}")

y_test_pred_svc = svc_model.predict(X_test_w2v)
print("\n[SVC] Test Set:")
print(classification_report(y_test, y_test_pred_svc))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_svc):.4f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_test_pred_svc, average='weighted'):.4f}")


from sklearn.preprocessing import LabelEncoder

# Inisialisasi dan fit encoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train_resampled)

# Transform label menjadi integer
y_train_enc = label_encoder.transform(y_train_resampled)
y_val_enc = label_encoder.transform(y_val)
y_test_enc = label_encoder.transform(y_test)

# ========================
# ========================
# Model 3: XGBoost
# ========================
print("\nMelatih XGBoost Classifier...")
start_time = time.time()

xgb_model = xgb.XGBClassifier(objective='multi:softmax',
                              num_class=3,
                              eval_metric='mlogloss',
                              use_label_encoder=False,
                              n_estimators=100,
                              random_state=42,
                              n_jobs=-1)

xgb_model.fit(X_train_w2v_resampled, y_train_enc)
end_time = time.time()
print(f"Waktu pelatihan XGBoost: {end_time - start_time:.2f} detik")

# Prediksi (hasil berupa angka), konversi ke string label
# y_val_pred_xgb_enc = xgb_model.predict(X_val_w2v)
y_val_pred_xgb = label_encoder.inverse_transform(y_val_pred_xgb_enc)

# print("\n[XGBoost] Validation Set:")
# print(classification_report(y_val, y_val_pred_xgb))
# print(f"Accuracy: {accuracy_score(y_val, y_val_pred_xgb):.4f}")
# print(f"F1-Score (weighted): {f1_score(y_val, y_val_pred_xgb, average='weighted'):.4f}")

y_test_pred_xgb_enc = xgb_model.predict(X_test_w2v)
y_test_pred_xgb = label_encoder.inverse_transform(y_test_pred_xgb_enc)

print("\n[XGBoost] Test Set:")
print(classification_report(y_test, y_test_pred_xgb))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_xgb):.4f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_test_pred_xgb, average='weighted'):.4f}")




--- Melatih dan Mengevaluasi Model Klasifikasi (Word2Vec Lokal) ---

Melatih Random Forest Classifier...
Waktu pelatihan RF: 14.36 detik

[Random Forest] Test Set:
              precision    recall  f1-score   support

     negatif       0.58      0.56      0.57       519
      netral       0.65      0.60      0.62       578
     positif       0.69      0.74      0.72       882

    accuracy                           0.65      1979
   macro avg       0.64      0.63      0.64      1979
weighted avg       0.65      0.65      0.65      1979

Accuracy: 0.6534
F1-Score (weighted): 0.6517

Melatih Support Vector Classifier (SVC)...
Waktu pelatihan SVC: 15.43 detik

[SVC] Test Set:
              precision    recall  f1-score   support

     negatif       0.66      0.19      0.30       519
      netral       0.36      0.62      0.45       578
     positif       0.61      0.57      0.59       882

    accuracy                           0.49      1979
   macro avg       0.54      0.46      0.45

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:40:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Waktu pelatihan XGBoost: 15.54 detik

[XGBoost] Test Set:
              precision    recall  f1-score   support

     negatif       0.61      0.62      0.62       519
      netral       0.62      0.58      0.60       578
     positif       0.72      0.74      0.73       882

    accuracy                           0.66      1979
   macro avg       0.65      0.65      0.65      1979
weighted avg       0.66      0.66      0.66      1979

Accuracy: 0.6625
F1-Score (weighted): 0.6618


In [19]:
rf_model_loaded = joblib.load('model_random_forest_tfidf.pkl')
svc_model_loaded = joblib.load('model_svc_tfidf.pkl')
xgb_model_loaded = joblib.load('model_xgboost_tfidf.pkl')
label_encoder_loaded = joblib.load('label_encoder_tfidf.pkl')


In [20]:
# Dictionary kalimat representatif tiap label
contoh_kalimat = {
    "Positif": [
        "Saya sangat senang dengan pelayanan ini.",
        "Produk ini luar biasa bagus dan berkualitas.",
        "Saya merasa puas dan akan membeli lagi."
    ],
    "Netral": [
        "Saya baru saja mencoba produk ini.",
        "Pengirimannya sesuai dengan yang dijanjikan.",
        "Saya belum tahu apakah produk ini bagus atau tidak."
    ],
    "Negatif": [
        "Saya kecewa dengan kualitasnya.",
        "Pelayanannya sangat lambat dan tidak ramah.",
        "Saya tidak akan membeli produk ini lagi."
    ]
}


In [21]:
# Himpun semua kalimat uji dari dictionary
kalimat_uji = []
label_asli = []

for label, kalimat_list in contoh_kalimat.items():
    for kalimat in kalimat_list:
        kalimat_uji.append(kalimat)
        label_asli.append(label)

# Transformasikan kalimat uji ke TF-IDF
X_kalimat_uji_vec = vectorizer.transform(kalimat_uji)


In [22]:
# Random Forest
pred_rf = rf_model.predict(X_kalimat_uji_vec)

# SVC
pred_svc = svc_model.predict(X_kalimat_uji_vec)

# XGBoost
pred_xgb_encoded = xgb_model.predict(X_kalimat_uji_vec)
pred_xgb = label_encoder.inverse_transform(pred_xgb_encoded)
print("\n=== Interpretasi Prediksi Model ===")
for i, kalimat in enumerate(kalimat_uji):
    print(f"\nKalimat: {kalimat}")
    print(f"Label Asli: {label_asli[i]}")
    print(f"Prediksi Random Forest: {pred_rf[i]}")
    print(f"Prediksi SVC: {pred_svc[i]}")
    print(f"Prediksi XGBoost: {pred_xgb[i]}")



=== Interpretasi Prediksi Model ===

Kalimat: Saya sangat senang dengan pelayanan ini.
Label Asli: Positif
Prediksi Random Forest: netral
Prediksi SVC: netral
Prediksi XGBoost: netral

Kalimat: Produk ini luar biasa bagus dan berkualitas.
Label Asli: Positif
Prediksi Random Forest: positif
Prediksi SVC: positif
Prediksi XGBoost: positif

Kalimat: Saya merasa puas dan akan membeli lagi.
Label Asli: Positif
Prediksi Random Forest: positif
Prediksi SVC: positif
Prediksi XGBoost: positif

Kalimat: Saya baru saja mencoba produk ini.
Label Asli: Netral
Prediksi Random Forest: positif
Prediksi SVC: positif
Prediksi XGBoost: positif

Kalimat: Pengirimannya sesuai dengan yang dijanjikan.
Label Asli: Netral
Prediksi Random Forest: positif
Prediksi SVC: positif
Prediksi XGBoost: positif

Kalimat: Saya belum tahu apakah produk ini bagus atau tidak.
Label Asli: Netral
Prediksi Random Forest: positif
Prediksi SVC: positif
Prediksi XGBoost: positif

Kalimat: Saya kecewa dengan kualitasnya.
Label Asl